# Introdução

Sistemas de recomendação são ferramentas que filtram e processam informações, retornando um resultado que faça sentido para o contexto em que está sendo aplicado. Esses sistemas utilizam algoritmos específicos para oferecer sugestões personalizadas, facilitando a tomada de decisão do usuário. Neste estudo, buscamos compreender os principais algoritmos empregados em sistemas de recomendação e avaliar seu desempenho aplicando-os ao "Book-Crossing Dataset", um conjunto de dados que reúne informações sobre livros, avaliações e interações de usuários.

A partir dessa análise, pretendemos identificar os métodos mais eficazes para realizar recomendações precisas dentro deste contexto.

# Materiais e Métodos:

## Dependências

As bibliotecas necessárias para rodar os nossos testes de caso.

In [1]:
%pip install kagglehub
%pip install pandas
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import os

/usr/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## DataSet

Nós escolhemos o Book-Crossing Dataset, que relaciona usuários, livros e avaliações, permitindo analisar padrões de leitura e preferências. Com isso, aplicaremos algoritmos de recomendação para sugerir livros personalizados, baseados em interações anteriores.

### Importa DataSet

In [ ]:
path = kagglehub.dataset_download("somnambwl/bookcrossing-dataset")

books = pd.read_csv(path+"/Books.csv", sep=";", header=0)
users = pd.read_csv(path+"/Users.csv", sep=";", low_memory=False, header=0)
ratings = pd.read_csv(path+"/Ratings.csv", sep=";", low_memory=False, header=0)

### Visualização dos dados importados

In [ ]:
books.head()

In [ ]:
users.head()

In [ ]:
ratings.head()